# User Manual
## Installation
1. Ensure Python 3.x is installed on your system.
2. Install required dependencies using `pip install bcrypt`.
3. Save the provided code as a `.ipynb` file or run it directly in a Jupyter Notebook.

## Usage
1. Run the code cells in sequence or execute the main script.
2. Choose between registering a new user or logging in.
3. Use the menu options to:
   - Add transactions.
   - View transaction history.
   - Exit the application.

## Notes
- Ensure the `finance_manager.db` database file is in the same directory as the script.
- Backup and restore functionality is available for data security.

## Error Handling
The application includes error handling for common issues like invalid inputs and database errors. Please report unhandled issues to the development team.

 # User Registration and Authentication 
# Objectives:

- Implement user registration with unique usernames and passwords.
- Add login functionality to authenticate users.
# Approach:

- Use Python's sqlite3 to store users' data.
- Store passwords securely using hashing (e.g., bcrypt or hashlib).
# Steps:

- Database Setup:

- Create a table users with columns for username, password_hash, and email.
- User Registration:

- Take input from the user for username, password, and email.
- Hash the password before storing it in the database.
# Login Functionality:

- Verify the username and password by comparing the hashed version of the entered password with the stored hash

In [1]:
import sqlite3
from tabulate import tabulate
import bcrypt

def create_user_table():
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS users
                      (id INTEGER PRIMARY KEY, username TEXT, password_hash TEXT, email TEXT)''')
    conn.commit()
    conn.close()

def register_user(username, password, email):
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    password_hash = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())
    cursor.execute('INSERT INTO users (username, password_hash, email) VALUES (?, ?, ?)', (username, password_hash, email))
    conn.commit()
    conn.close()

def authenticate_user(username, password):
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    cursor.execute('SELECT password_hash FROM users WHERE username = ?', (username,))
    result = cursor.fetchone()
    conn.close()
    if result and bcrypt.checkpw(password.encode('utf-8'), result[0]):
        return True
    return False


# Income and Expense Tracking 
# Objectives:

- Allow users to add, update, and delete income and expense entries.
- Categorize transactions (e.g., Food, Rent, Salary).
# Approach:

- Use income and expense tables with columns like amount, category, date, type (income/expense).
- Provide functions for CRUD operations: Add, Update, Delete.
# Steps:

- Create income and expenses tables.
- Add methods for adding, editing, and deleting entries.
- Display all entries with filters by type and category.

In [2]:
def create_transactions_table():
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS transactions
                      (id INTEGER PRIMARY KEY, user_id INTEGER, amount REAL, category TEXT, date TEXT, type TEXT)''')
    conn.commit()
    conn.close()

def add_transaction(user_id, amount, category, date, transaction_type):
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    cursor.execute('INSERT INTO transactions (user_id, amount, category, date, type) VALUES (?, ?, ?, ?, ?)', 
                   (user_id, amount, category, date, transaction_type))
    conn.commit()
    conn.close()

def update_transaction(transaction_id, amount, category, date, transaction_type):
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    cursor.execute('''UPDATE transactions 
                      SET amount = ?, category = ?, date = ?, type = ? 
                      WHERE id = ?''', (amount, category, date, transaction_type, transaction_id))
    conn.commit()
    conn.close()

def delete_transaction(transaction_id):
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    cursor.execute('DELETE FROM transactions WHERE id = ?', (transaction_id,))
    conn.commit()
    conn.close()

def get_transactions(user_id, transaction_type=None):
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    if transaction_type:
        cursor.execute('SELECT * FROM transactions WHERE user_id = ? AND type = ?', (user_id, transaction_type))
    else:
        cursor.execute('SELECT * FROM transactions WHERE user_id = ?', (user_id,))
    transactions = cursor.fetchall()
    conn.close()
    return transactions


def get_user_id(username):
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    cursor.execute("SELECT id FROM users WHERE username = ?", (username,))
    user_id = cursor.fetchone()
    conn.close()
    
    if user_id:
        return user_id[0]
    else:
        return None  # Return None if the user doesn't exist


# Financial Reports 
# Objectives:

- Generate monthly and yearly financial reports.
- Calculate total income, expenses, and savings.
# Approach:

- Create queries to aggregate the sum of income and expenses for the required period (monthly, yearly).
- Calculate savings as the difference between income and expenses.

In [3]:
from datetime import datetime

def get_monthly_report(user_id, month, year):
    start_date = f"{year}-{month}-01"
    end_date = f"{year}-{month}-31"
    
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    
    # Get total income for the month
    cursor.execute('''SELECT SUM(amount) FROM transactions 
                      WHERE user_id = ? AND type = 'income' AND date BETWEEN ? AND ?''', 
                   (user_id, start_date, end_date))
    total_income = cursor.fetchone()[0] or 0
    
    # Get total expenses for the month
    cursor.execute('''SELECT SUM(amount) FROM transactions 
                      WHERE user_id = ? AND type = 'expense' AND date BETWEEN ? AND ?''', 
                   (user_id, start_date, end_date))
    total_expenses = cursor.fetchone()[0] or 0
    
    # Get all transactions for the month
    cursor.execute('''SELECT id, user_id, amount, category, date, type FROM transactions 
                      WHERE user_id = ? AND date BETWEEN ? AND ?''', 
                   (user_id, start_date, end_date))
    transactions = cursor.fetchall()
    
    conn.close()
    
    savings = total_income - total_expenses
    return total_income, total_expenses, savings, transactions

def get_yearly_report(user_id, year):
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"
    
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    
    # Get total income for the year
    cursor.execute('''SELECT SUM(amount) FROM transactions 
                      WHERE user_id = ? AND type = 'income' AND date BETWEEN ? AND ?''', 
                   (user_id, start_date, end_date))
    total_income = cursor.fetchone()[0] or 0
    
    # Get total expenses for the year
    cursor.execute('''SELECT SUM(amount) FROM transactions 
                      WHERE user_id = ? AND type = 'expense' AND date BETWEEN ? AND ?''', 
                   (user_id, start_date, end_date))
    total_expenses = cursor.fetchone()[0] or 0
    
    # Get all transactions for the year
    cursor.execute('''SELECT id, user_id, amount, category, date, type FROM transactions 
                      WHERE user_id = ? AND date BETWEEN ? AND ?''', 
                   (user_id, start_date, end_date))
    transactions = cursor.fetchall()
    
    conn.close()
    
    savings = total_income - total_expenses
    return total_income, total_expenses, savings, transactions


# Budgeting 
# Objectives:

- Enable users to set monthly budgets for different categories.
- Notify users when they exceed their budget limits.
# Approach:

- Create a table for storing budget limits by category and month.
- Compare the total expenses in each category against the set budget.

In [4]:
def create_budget_table():
    try:
        conn = sqlite3.connect('finance_manager.db')
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS budgets (
                user_id INTEGER,
                month INTEGER,
                year INTEGER,
                category TEXT,
                budget_limit REAL
            )
        ''')
        conn.commit()
        conn.close()
        print("Budgets table created successfully or already exists.")
    except sqlite3.Error as e:
        print(f"Error creating budgets table: {e}")



def set_budget(user_id, month, year, category, budget_limit):
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    cursor.execute('''REPLACE INTO budgets (user_id, month, year, category, budget_limit) 
                      VALUES (?, ?, ?, ?, ?)''', (user_id, month, year, category, budget_limit))
    conn.commit()
    conn.close()

def check_budget_exceed(user_id, month, year, category):
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    cursor.execute('SELECT budget_limit FROM budgets WHERE user_id = ? AND month = ? AND year = ? AND category = ?', 
                   (user_id, month, year, category))
    budget_limit = cursor.fetchone()
    if not budget_limit:
        return None
    budget_limit = budget_limit[0]
    cursor.execute('''SELECT SUM(amount) FROM transactions 
                      WHERE user_id = ? AND type = 'expense' AND category = ? 
                      AND date BETWEEN ? AND ?''', 
                   (user_id, category, f"{year}-{month}-01", f"{year}-{month}-31"))
    total_expenses = cursor.fetchone()[0] or 0
    conn.close()
    return total_expenses > budget_limit


# Data Persistence 
# Objectives:

- Store data and transactions in a database.
- Implement functions for backup and restore.
# Approach:

- Implement backup functionality using SQLite's .backup() method.
- Create a restore function to load a backup into the current database.# 

In [5]:
def backup_database():
    conn = sqlite3.connect('finance_manager.db')
    backup_conn = sqlite3.connect('finance_manager_backup.db')
    conn.backup(backup_conn)
    backup_conn.close()
    conn.close()

def restore_database():
    conn = sqlite3.connect('finance_manager.db')
    backup_conn = sqlite3.connect('finance_manager_backup.db')
    backup_conn.backup(conn)
    backup_conn.close()
    conn.close()


# Testing and Documentation 
# Objectives:

- Write unit tests for core functionalities.
- Create a user manual with installation and usage instructions.
# Approach:

- Use unittest to write unit tests for user registration, transaction management, report generation, etc.
- Provide a clear user manual explaining installation, configuration, and basic usage of the app.



In [7]:
import unittest
from unittest.mock import patch

class TestFinanceManager(unittest.TestCase):

    def test_user_registration(self):
        # Register a user and check authentication
        register_user('test_user', 'password123', 'test@example.com')
        self.assertTrue(authenticate_user('test_user', 'password123'))

    def test_user_registration_failure(self):
        # Test for failed authentication
        self.assertFalse(authenticate_user('test_user', 'wrong_password'))

    def test_add_transaction(self):
        # Add a transaction and verify it appears in the transactions list
        add_transaction(1, 100, 'Salary', '2024-12-01', 'income')
        transactions = get_transactions(1)
        self.assertEqual(len(transactions), 1)
        self.assertEqual(transactions[0][2], 100)  # Verify the amount is correct

    def test_get_transactions_empty(self):
        # Test for no transactions in the database
        transactions = get_transactions(1)
        self.assertEqual(len(transactions), 0)

    def test_monthly_report(self):
        # Assuming user_id 1 has data in the database
        total_income, total_expenses, savings = get_monthly_report(1, 12, 2024)
        self.assertIsInstance(total_income, float)
        self.assertIsInstance(total_expenses, float)
        self.assertIsInstance(savings, float)

    def test_yearly_report(self):
        # Assuming user_id 1 has data in the database
        total_income, total_expenses, savings = get_yearly_report(1, 2024)
        self.assertIsInstance(total_income, float)
        self.assertIsInstance(total_expenses, float)
        self.assertIsInstance(savings, float)

    @patch('sqlite3.connect')  # Mocking the SQLite connection for testing purposes
    def test_set_budget(self, mock_connect):
        # Mocking database interaction for set_budget
        mock_cursor = mock_connect.return_value.cursor.return_value
        set_budget(1, 12, 2024, 'Food', 500)
        mock_cursor.execute.assert_called_with(
            '''INSERT OR REPLACE INTO budget (user_id, month, year, category, budget_limit)
               VALUES (?, ?, ?, ?, ?)''', (1, 12, 2024, 'Food', 500))

    def test_check_budget_exceed(self):
        # Test when the budget limit is exceeded
        # Assuming user_id 1 has set a budget and has exceeded the limit
        exceeded = check_budget_exceed(1, 12, 2024, 'Food')
        self.assertTrue(exceeded)

    def test_check_budget_within_limit(self):
        # Test when the budget limit is not exceeded
        # Assuming user_id 1 has set a budget and is within the limit
        exceeded = check_budget_exceed(1, 12, 2024, 'Transport')
        self.assertFalse(exceeded)

    def test_invalid_budget_check(self):
        # Test when no budget is set for a category
        exceeded = check_budget_exceed(1, 12, 2024, 'NonExistentCategory')
        self.assertFalse(exceeded)



In [9]:
def main():
    try:
        # Setup (creating tables if they don't exist)
        create_user_table()
        create_transactions_table()
        create_budget_table()

        print("Welcome to the Personal Finance Manager!")
        action = input("Would you like to (r)egister or (l)ogin? (r/l): ").strip().lower()

        current_user_id = None  # Variable to store logged-in user ID

        if action == 'r':
            # Register a new user
            username = input("Enter a username: ").strip()
            password = input("Enter a password: ").strip()
            email = input("Enter your email: ").strip()
            register_user(username, password, email)
            print(f"User {username} registered successfully!")

        elif action == 'l':
            # Login existing user
            username = input("Enter your username: ").strip()
            password = input("Enter your password: ").strip()
            
            if authenticate_user(username, password):
                # Assuming authenticate_user returns the user's ID on success
                current_user_id = get_user_id(username)  # Fetch user ID after successful login
                print("User authenticated successfully!")
            else:
                print("Authentication failed!")
                return

        else:
            print("Invalid action selected! Exiting.")
            return

        # User-specific main menu
        while True:
            print("\nWhat would you like to do?")
            print("1. Add Transaction")
            print("2. View Transactions")
            print("3. Generate Financial Reports")
            print("4. Manage Budgeting")
            print("5. Exit")
            
            choice = input("Enter your choice (1-5): ").strip()

            if choice == '1':
                # Add Transaction
                amount = float(input("Enter the amount: "))
                category = input("Enter the category (e.g., Food, Salary): ").strip()
                date = input("Enter the transaction date (YYYY-MM-DD): ").strip()
                transaction_type = input("Enter transaction type (income/expense): ").strip().lower()

                if transaction_type not in ['income', 'expense']:
                    print("Invalid transaction type! Please enter 'income' or 'expense'.")
                    continue

                if current_user_id is None:
                    print("No user is logged in!")
                    continue

                add_transaction(current_user_id, amount, category, date, transaction_type)
                print(f"Transaction of {amount} ({transaction_type}) for {category} added.")

            elif choice == '2':
                # View Transactions
                if current_user_id is None:
                    print("No user is logged in!")
                    continue

                transactions = get_transactions(current_user_id)
                if not transactions:
                    print("No transactions found.")
                else:
                    print("Transactions:")
                    headers = ["ID", "User ID", "Amount", "Category", "Date", "Type"]
                    print(tabulate(transactions, headers=headers, tablefmt="grid"))

            elif choice == '3':
                # Generate Financial Reports
                print("1. Monthly Report")
                print("2. Yearly Report")
                report_choice = input("Enter your choice (1-2): ").strip()

                if report_choice == '1':
                    month = input("Enter month (MM): ").strip()
                    year = input("Enter year (YYYY): ").strip()
                    total_income, total_expenses, savings, transactions = get_monthly_report(current_user_id, month, year)
                    print("\nMonthly Report:")
                    print(tabulate(transactions, headers=["ID", "User ID", "Amount", "Category", "Date", "Type"], tablefmt="grid"))
                    print(f"Summary: Total Income: {total_income}, Total Expenses: {total_expenses}, Savings: {savings}")
                elif report_choice == '2':
                    year = input("Enter year (YYYY): ").strip()
                    total_income, total_expenses, savings, transactions = get_yearly_report(current_user_id, year)
                    print("\nYearly Report:")
                    print(tabulate(transactions, headers=["ID", "User ID", "Amount", "Category", "Date", "Type"], tablefmt="grid"))
                    print(f"Summary: Total Income: {total_income}, Total Expenses: {total_expenses}, Savings: {savings}")

            elif choice == '4':
                # Manage Budgeting
                print("1. Set Budget")
                print("2. Check Budget Exceedance")
                budget_choice = input("Enter your choice (1-2): ").strip()

                if budget_choice == '1':
                    month = input("Enter month (MM): ").strip()
                    year = input("Enter year (YYYY): ").strip()
                    category = input("Enter category: ").strip()
                    limit = float(input("Enter budget limit: "))
                    if current_user_id is None:
                        print("No user is logged in!")
                        continue
                    set_budget(current_user_id, int(month), int(year), category, limit)
                    print(f"Budget set for {category} in {month}/{year}.")
                elif budget_choice == '2':
                    month = input("Enter month (MM): ").strip()
                    year = input("Enter year (YYYY): ").strip()
                    category = input("Enter category: ").strip()
                    if current_user_id is None:
                        print("No user is logged in!")
                        continue
                    if check_budget_exceed(current_user_id, int(month), int(year), category):
                        print(f"Budget exceeded for {category} in {month}/{year}.")
                    else:
                        print(f"Budget is within the limit for {category} in {month}/{year}.")

            elif choice == '5':
                # Exit the application
                print("Exiting the application. Goodbye!")
                break

            else:
                print("Invalid choice! Please select between 1-5.")

    except Exception as e:
        print(f"An error occurred: {e}")




Budgets table created successfully or already exists.
Welcome to the Personal Finance Manager!


Authentication failed!


In [10]:
main()

Budgets table created successfully or already exists.
Welcome to the Personal Finance Manager!
Authentication failed!
